In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.2'

In [20]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Dropout, Reshape

In [5]:
import numpy as np

In [ ]:
# (24, 4096, 9, 1)

In [6]:
# create random matrixinput
x_train = np.random.random(size=(24,4096,9,1))

In [9]:
# construct net
# conv(64)->conv(64)->conv(64)->conv(128)
# define input
input_pc = Input(shape=(4096,9,1))
# 1st conv
x = Conv2D(filters=64, kernel_size=(1,9), padding='valid', activation='relu')(input_pc)

In [10]:
# 2nd conv
x = Conv2D(filters=64, kernel_size=(1,1), padding='valid', activation='relu')(x)
# 3rd conv
x = Conv2D(filters=64, kernel_size=(1,1), padding='valid', activation='relu')(x)
# 4th conv
x = Conv2D(filters=128, kernel_size=(1,1), padding='valid', activation='relu')(x)

In [15]:
# 5th conv, get points_feat1
points_feat1 = Conv2D(filters=1024, kernel_size=(1,1), padding='valid', activation='relu')(x)

In [16]:
# MAX-pooling, obtain  global feature for point_cloud, kernel_size=[4096,1]
# "global_feature"
pc_feat1 = MaxPool2D(pool_size=(4096,1), padding="valid")(points_feat1)

In [18]:
# Fully-Connected, batch_size=24
#pc_feat1 = tf.reshape(pc_feat1, [batch_size, -1]) # similar to flatten the whole batch
# flatten:
pc_feat1 = Flatten()(pc_feat1)
# fc1
pc_feat1 = Dense(units=256, activation='relu')(pc_feat1)
# fc2
pc_feat1 = Dense(units=128, activation='relu')(pc_feat1)

#pc_feat1 = tf_util.fully_connected(pc_feat1, 256, bn=True, is_training=is_training, scope='fc1', bn_decay=bn_decay)
#pc_feat1 = tf_util.fully_connected(pc_feat1, 128, bn=True, is_training=is_training, scope='fc2', bn_decay=bn_decay)

In [26]:
??keras.backend.tile

In [29]:
pc_feat1_expand_tmp = keras.backend.tile(pc_feat1_expand, (1,4096,1,1))

In [21]:
# CONCAT 
# reshape back to (1,1,128)
pc_feat1_expand = Reshape((1,1,-1))(pc_feat1)
pc_feat1_expand = keras.backend.tile(pc_feat1_expand, (1,4096, 1, 1))

TypeError: __init__() takes exactly 2 arguments (1 given)

In [40]:
pc_feat1_expand = keras.backend.tile(pc_feat1_expand, (1,4096, 1, 1))

In [ ]:
# CONCAT 
# reshape back to (1,1,128)
pc_feat1_expand = Reshape()(pc_feat1, [1,1,-1])
pc_feat1_expand = keras.backend.tile(pc_feat1_expand, [4096, 1, 1])
#pc_feat1_expand = tf.tile(tf.reshape(pc_feat1, [batch_size, 1, 1, -1]), [1, num_point, 1, 1])
#points_feat1_concat = tf.concat(axis=3, values=[points_feat1, pc_feat1_expand])

In [11]:
model_0 = Model(input_pc, x)

In [14]:
model_0.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 4096, 9, 1)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4096, 1, 64)       640       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4096, 1, 64)       4160      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4096, 1, 64)       4160      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4096, 1, 128)      8320      
Total params: 17,280.0
Trainable params: 17,280.0
Non-trainable params: 0.0
_________________________________________________________________


In [17]:
model_1 = Model(input_pc, pc_feat1)
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 4096, 9, 1)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4096, 1, 64)       640       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4096, 1, 64)       4160      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4096, 1, 64)       4160      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4096, 1, 128)      8320      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4096, 1, 1024)     132096    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 1024)        0         
Total para

In [19]:
model_2 = Model(input_pc, pc_feat1)
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 4096, 9, 1)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4096, 1, 64)       640       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4096, 1, 64)       4160      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4096, 1, 64)       4160      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4096, 1, 128)      8320      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4096, 1, 1024)     132096    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 1024)        0         
__________

In [35]:
type(input_pc), input_pc.shape

(tensorflow.python.framework.ops.Tensor,
 TensorShape([Dimension(None), Dimension(4096), Dimension(9), Dimension(1)]))

In [42]:
import keras.backend as K

In [38]:
type(pc_feat1_expand), pc_feat1_expand.shape

(tensorflow.python.framework.ops.Tensor,
 TensorShape([Dimension(None), Dimension(1), Dimension(1), Dimension(128)]))

In [37]:
type(pc_feat1_expand_tmp), pc_feat1_expand_tmp.shape

(tensorflow.python.framework.ops.Tensor,
 TensorShape([Dimension(None), Dimension(4096), Dimension(1), Dimension(128)]))

In [41]:
model_3 = Model(input_pc, pc_feat1_expand)
model_3.summary()

TypeError: Output tensors to a Model must be Keras tensors. Found: Tensor("Tile_4:0", shape=(?, 4096, 1, 128), dtype=float32)

In [ ]:
pc_feat1 = tf_util.max_pool2d(inputs=points_feat1, kernel_size=[num_point,1], padding='VALID', scope='maxpool1')

In [4]:
points_feat1 = tf_util.conv2d(inputs=net, num_output_channels=1024, kernel_size=[1,1], padding='VALID', stride=[1,1],
                     bn=True, is_training=is_training, scope='conv5', bn_decay=bn_decay)